![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.7.Clinical_Deidentification_in_Portuguese.ipynb)

# Clinical Deidentification in Portuguese

**Protected Health Information**:

Individual’s past, present, or future physical or mental health or condition
provision of health care to the individual
past, present, or future payment for the health care
Protected health information includes many common identifiers (e.g., name, address, birth date, Social Security Number) when they can be associated with the health information.

In [1]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare-3.5.0.json to spark_nlp_for_healthcare-3.5.0.json


In [2]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

     |████████████████████████████████| 212.4 MB 70 kB/s 
     |████████████████████████████████| 198 kB 42.9 MB/s 
     |████████████████████████████████| 188 kB 2.2 MB/s 
     |████████████████████████████████| 142 kB 5.1 MB/s 


In [3]:
import sys
import os
import json
import pandas as pd
import string
import numpy as np

import sparknlp
import sparknlp_jsl

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import ResourceDownloader
from sparknlp.util import *
from sparknlp_jsl.annotator import *

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(SECRET, params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.5.0


# 1. Italian NER Deidentification Models
We have two different models you can use:
* `ner_deid_generic`, detects 8 entities
* `ner_deid_subentity`, detects 19 entities

### Creating pipeline

In [4]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d", "pt")\
    .setInputCols(["document","token"])\
	  .setOutputCol("embeddings")

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
w2v_cc_300d download started this may take some time.
Approximate size to download 1.1 GB
[OK!]


## 1.1. NER Deid Generic

**`ner_deid_generic`** extracts:
- Name
- Profession
- Age
- Date
- Contact (Telephone numbers, Email addresses)
- Location (Address, City, Postal code, Hospital Name, Organization)
- ID (Social Security numbers, Medical record numbers)
- Sex

In [5]:
ner_generic = MedicalNerModel.pretrained("ner_deid_generic", "pt", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_generic")

ner_converter_generic = NerConverter()\
    .setInputCols(["sentence","token","ner_deid_generic"])\
    .setOutputCol("ner_chunk_generic")

ner_deid_generic download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [6]:
ner_generic.getClasses()

['O',
 'I-LOCATION',
 'I-CONTACT',
 'I-PROFESSION',
 'I-NAME',
 'I-DATE',
 'B-ID',
 'B-PROFESSION',
 'B-CONTACT',
 'I-ID',
 'B-NAME',
 'B-DATE',
 'B-LOCATION',
 'B-SEX',
 'I-SEX',
 'B-AGE']

## 1.2. NER Deid Subentity

**`ner_deid_subentity`** extracts:

`PATIENT`, `HOSPITAL`, `DATE`, `ORGANIZATION`, `CITY`, `ID`, `STREET`, `SEX`, `EMAIL`, `ZIP`, `PROFESSION`, `PHONE`, `COUNTRY`, `DOCTOR`, `AGE`

In [7]:
ner_subentity = MedicalNerModel.pretrained("ner_deid_subentity", "pt", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_subentity")

ner_converter_subentity = NerConverter()\
    .setInputCols(["sentence", "token", "ner_deid_subentity"])\
    .setOutputCol("ner_chunk_subentity")

ner_deid_subentity download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [8]:
ner_subentity.getClasses()

['O',
 'B-ORGANIZATION',
 'I-PROFESSION',
 'B-DOCTOR',
 'B-PROFESSION',
 'I-ID',
 'B-CITY',
 'B-DATE',
 'B-PATIENT',
 'B-SEX',
 'I-SEX',
 'I-DOCTOR',
 'I-CITY',
 'I-DATE',
 'B-COUNTRY',
 'B-ID',
 'B-ZIP',
 'I-STREET',
 'I-PATIENT',
 'B-PHONE',
 'I-PHONE',
 'B-HOSPITAL',
 'B-EMAIL',
 'B-STREET',
 'I-ORGANIZATION',
 'I-HOSPITAL',
 'B-AGE',
 'I-COUNTRY']

## 1.3. Pipeline

In [91]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_generic,
      ner_converter_generic,
      ner_subentity,
      ner_converter_subentity,
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [97]:
text = """Detalhes do paciente.
Nome do paciente:  Pedro Gonçalves
NHC: 2569870.
Endereço: Rua Das Flores 23.
Código Postal: 21754-987.
Dados de cuidados.
Data de nascimento: 10/10/1963.
Idade: 53 anos 
Data de admissão: 17/06/2016.
Doutora: Maria Santos"""

text_df = spark.createDataFrame([[text]]).toDF("text")

result = model.transform(text_df)

### Results for `ner_generic`

In [98]:
result.select(F.explode(F.arrays_zip('ner_chunk_generic.result', 'ner_chunk_generic.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------+---------+
|chunk            |ner_label|
+-----------------+---------+
|Pedro Gonçalves  |NAME     |
|2569870          |ID       |
|Rua Das Flores 23|LOCATION |
|21754-987        |LOCATION |
|10/10/1963       |DATE     |
|53               |AGE      |
|17/06/2016       |DATE     |
|Maria Santos     |NAME     |
+-----------------+---------+



### Results for `ner_subentity`

In [99]:
result.select(F.explode(F.arrays_zip('ner_chunk_subentity.result', 'ner_chunk_subentity.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------+---------+
|chunk            |ner_label|
+-----------------+---------+
|Pedro Gonçalves  |PATIENT  |
|2569870          |ID       |
|Rua Das Flores 23|STREET   |
|21754-987        |ZIP      |
|10/10/1963       |DATE     |
|53               |AGE      |
|17/06/2016       |DATE     |
|Maria Santos     |DOCTOR   |
+-----------------+---------+



## DeIdentification

### Obfuscation mode

In [100]:
# Downloading faker entity list.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/obfuscate_pt.txt

In [101]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)

deid_obfuscated = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('obfuscate_pt.txt')\
    .setObfuscateRefSource("file")

In [102]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [103]:
deid_lp = LightPipeline(model)

In [119]:
text = """Detalhes do paciente.
Nome do paciente: Antonio Gonçalves
NHC: 2569870.
Endereço: Rua Das Flores 23.
Código Postal: 21754-987.
Dados de cuidados.
Data de nascimento: 10/10/1963.
Idade: 23 anos 
Data de admissão: 17/06/2016.
Doutora: Maria Santos"""

In [120]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

Detalhes do paciente.
Nome do paciente: <PATIENT>
NHC: <ID>.
Endereço: <STREET>.
Código Postal: <ZIP>.
Dados de cuidados.
Data de nascimento: <DATE>.
Idade: <AGE> anos 
Data de admissão: <DATE>.

Doutora: <DOCTOR>


Detalhes do paciente.
Nome do paciente: [***************]
NHC: [*****].
Endereço: [***************].
Código Postal: [*******].
Dados de cuidados.
Data de nascimento: [********].
Idade: ** anos 
Data de admissão: [********].

Doutora: [**********]


Detalhes do paciente.
Nome do paciente: ****
NHC: ****.
Endereço: ****.
Código Postal: ****.
Dados de cuidados.
Data de nascimento: ****.
Idade: **** anos 
Data de admissão: ****.

Doutora: ****


Detalhes do paciente.
Nome do paciente: Lourenço Morais
NHC: 569 4653.
Endereço: R. Cuf, 304.
Código Postal: 74913712.
Dados de cuidados.
Data de nascimento: 28/11/1963.
Idade: 53 anos 
Data de admissão: 24/06/2016.

Doutora: Moreira


In [121]:
pd.set_option("display.max_colwidth", 200)

df = pd.DataFrame(list(zip(result["masked_with_entity"], 
                           result["masked_with_chars"],
                           result["masked_fixed_length_chars"], 
                           result["obfuscated"])),
                 columns= ["Masked_with_entity", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked_with_entity,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,Detalhes do paciente.,Detalhes do paciente.,Detalhes do paciente.,Detalhes do paciente.
1,Nome do paciente: <PATIENT>,Nome do paciente: [***************],Nome do paciente: ****,Nome do paciente: Lourenço Morais
2,NHC: <ID>.,NHC: [*****].,NHC: ****.,NHC: 569 4653.
3,Endereço: <STREET>.\nCódigo Postal: <ZIP>.,Endereço: [***************].\nCódigo Postal: [*******].,Endereço: ****.\nCódigo Postal: ****.,"Endereço: R. Cuf, 304.\nCódigo Postal: 74913712."
4,Dados de cuidados.,Dados de cuidados.,Dados de cuidados.,Dados de cuidados.
5,Data de nascimento: <DATE>.,Data de nascimento: [********].,Data de nascimento: ****.,Data de nascimento: 28/11/1963.
6,Idade: <AGE> anos,Idade: ** anos,Idade: **** anos,Idade: 53 anos
7,Data de admissão: <DATE>.,Data de admissão: [********].,Data de admissão: ****.,Data de admissão: 24/06/2016.
8,\nDoutora: <DOCTOR>,\nDoutora: [**********],\nDoutora: ****,\nDoutora: Moreira


# 2. Pretrained Portuguese Deidentification Pipeline

- We developed a clinical deidentification pretrained pipeline that can be used to deidentify PHI information from Italian medical texts. The PHI information will be masked and obfuscated in the resulting text. 
- The pipeline can mask and obfuscate:
    - Patient
    - Doctor
    - Hospital
    - Date
    - Organization
    - Sex
    - City
    - Street
    - Country
    - ZIP
    - Username
    - Profession
    - Phone
    - Email
    - Age
    - ID number
    - Medical record number
    - Account number
    - SSN
    - Plate Number
    - IP address
    - URL

In [127]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline("clinical_deidentification", "pt", "clinical/models")

clinical_deidentification download started this may take some time.
Approx size to download 1.2 GB
[OK!]


In [133]:
text = """RELAÇÃO HOSPITALAR
NOME: Pedro Gonçalves
NHC: MVANSK92F09W408A
ENDEREÇO: Rua Burcardo 7
CÓDIGO POSTAL: 80139
DATA DE NASCIMENTO: 03/03/1946
IDADE: 70 anos
SEXO: Homens
E-MAIL: pgon21@tim.pt
DATA DE ADMISSÃO: 12/12/2016
DOUTORA: Eva Andrade
RELATO CLÍNICO: 70 anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia.
Ele foi encaminhado a nós por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.
O exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV.
A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.
O hemograma é normal; a bioquímica mostrou uma glicemia de 169 mg/dl e triglicerídeos 456 mg/dl; função hepática e renal são normais. PSA de 1,16 ng/ml.

DIRIGIDA A: Dr. Eva Andrade - Centro Hospitalar do Medio Ave - Avenida Dos Aliados, 56
E-MAIL: evandrade@poste.pt
"""

result = deid_pipeline.annotate(text)
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

RELAÇÃO HOSPITALAR
NOME: [*************]
NHC: [**************]
ENDEREÇO: [************]
CÓDIGO POSTAL: [***]
DATA DE NASCIMENTO: [********]
IDADE: ** anos
SEXO: [****]
E-MAIL: [***********]
DATA DE ADMISSÃO: [********]
DOUTORA: [*********]
RELATO CLÍNICO: ** anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia.
Ele foi encaminhado a nós por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.
O exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV.
A urinálise mostrou 4 hemácias/campo e 0-5 leucócitos/campo; o resto do sedimento era normal.
O hemograma é normal; a bioquímica mo

The results can also be inspected vertically by creating a Pandas dataframe as such:

In [134]:
pd.set_option("display.max_colwidth", None)

df = pd.DataFrame(list(zip(result["sentence"], 
                           result["masked"],
                           result["masked_with_chars"], 
                           result["masked_fixed_length_chars"], 
                           result["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,RELAÇÃO HOSPITALAR\nNOME: Pedro Gonçalves,RELAÇÃO HOSPITALAR\nNOME: <DOCTOR>,RELAÇÃO HOSPITALAR\nNOME: [*************],RELAÇÃO HOSPITALAR\nNOME: ****,RELAÇÃO HOSPITALAR\nNOME: Isabel Magalhães
1,NHC: MVANSK92F09W408A,NHC: <ID>,NHC: [**************],NHC: ****,NHC: 124 445 311
2,ENDEREÇO: Rua Burcardo 7,ENDEREÇO: <STREET>,ENDEREÇO: [************],ENDEREÇO: ****,"ENDEREÇO: Rua de Santa María, 100"
3,CÓDIGO POSTAL: 80139\nDATA DE NASCIMENTO: 03/03/1946,CÓDIGO POSTAL: <ZIP>\nDATA DE NASCIMENTO: <DATE>,CÓDIGO POSTAL: [***]\nDATA DE NASCIMENTO: [********],CÓDIGO POSTAL: ****\nDATA DE NASCIMENTO: ****,CÓDIGO POSTAL: 1000-306\nDATA DE NASCIMENTO: 17/04/1946
4,IDADE: 70 anos,IDADE: <AGE> anos,IDADE: ** anos,IDADE: **** anos,IDADE: 46 anos
5,SEXO: Homens,SEXO: <SEX>,SEXO: [****],SEXO: ****,SEXO: Mulher
6,E-MAIL: pgon21@tim.pt\nDATA DE ADMISSÃO: 12/12/2016,E-MAIL: <EMAIL>\nDATA DE ADMISSÃO: <DATE>,E-MAIL: [***********]\nDATA DE ADMISSÃO: [********],E-MAIL: ****\nDATA DE ADMISSÃO: ****,E-MAIL: eric.shannon@geegle.com\nDATA DE ADMISSÃO: 26/12/2016
7,DOUTORA: Eva Andrade,DOUTORA: <DOCTOR>,DOUTORA: [*********],DOUTORA: ****,DOUTORA: Isabel Magalhães
8,"RELATO CLÍNICO: 70 anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia.","RELATO CLÍNICO: <AGE> anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia.","RELATO CLÍNICO: ** anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia.","RELATO CLÍNICO: **** anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia.","RELATO CLÍNICO: 46 anos, aposentado, sem alergia a medicamentos conhecida, com a seguinte história: ex-acidente de trabalho com fratura de vértebras e costelas; operado de doença de Dupuytren na mão direita e ponte ílio-femoral esquerda; diabetes tipo II, hipercolesterolemia e hiperuricemia; alcoolismo ativo, fuma 20 cigarros/dia."
9,"Ele foi encaminhado a nós por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV.","Ele foi encaminhado a nós por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV.","Ele foi encaminhado a nós por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado geral, com abdome e genitais normais; o toque retal foi compatível com adenoma de próstata grau I/IV.","Ele foi encaminhado a nós por apresentar hematúria macroscópica pós-evacuação em uma ocasião e microhematúria persistente posteriormente, com evacuação normal.\nO exame físico mostrou bom estado